In [ ]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import itertools
# Setting seed for reproducibility
np.random.seed(1234)  
PYTHONHASHSEED = 0

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,load_model,model_from_json
from keras.layers import Dense, Dropout, LSTM,BatchNormalization,TimeDistributed,Flatten,Bidirectional
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
data=pd.read_csv('theta/thetaBalanced.csv',header=None)
data.head()

### Divide the data into samples and labels

In [ ]:
data=check.iloc[0:,:-1]
labels=check.iloc[:,-1]
data.head()
data.shape
# labels.reshape(-1, 1)
labels=np.array(labels)
labels=labels.reshape((labels.shape[0],1))
labels.shape
# labels.head()

In [ ]:
labels=pd.DataFrame(labels)
labels.head()
data.head()

In [ ]:
def create_set(da, lab, window):
    seq, seq_lab=[],[]
    for k in range(0, len(da)-window, 20):
        q=lab.iloc[k:k+window]
        val=np.unique(q)
        if(len(val)==1):
            seq.append(np.array(da.iloc[k:k+window]))
#             print(f'sequence created with shape {np.array(seq).shape}')
            seq_lab.append(np.array(lab.iloc[k, 0]))
#             print(f'label appended {lab.iloc[k, 0]}')
        
    return seq, seq_lab

In [ ]:
window=32
seq, seq_labels=create_set(data, labels, window)

In [ ]:
X = np.asarray(seq).astype(np.float32)
Y = np.asarray(seq_labels)

In [ ]:
X.shape
Y.shape
Y=Y.reshape(Y.shape[0],1)
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(X, Y, train_size=0.80, shuffle=True)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


In [ ]:
np.unique(y_train,return_counts=True)
np.unique(y_test,return_counts=True)

In [ ]:
import tensorflow as tf
from keras import Sequential 
from keras.layers import Dense, Dropout, Flatten, LeakyReLU
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization
from tensorflow.python.keras.optimizers import *
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam

#from keras.optimizers import RMSprop 
#from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras import regularizers

# from tensorflow.python.keras.layers import Input, GRU, Dense
from tensorflow.python.keras.models import Model

In [ ]:
nb_features = x_train.shape[2]
nb_out = 1

model = Sequential()

model.add(LSTM( units=32,return_sequences=True,input_shape=(window, nb_features)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM( units=16,turn_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(units=8,activation='sigmoid'))
model.add(Dense(units=nb_out, activation='sigmoid'))
Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
model.fit(x_train, y_train,validation_split=0.2,epochs=30, batch_size=32,verbose=1,callbacks=[callback])

In [ ]:
Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.evaluate(x_test, y_test, verbose=1, batch_size=32)
scores[1]

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    fig=plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig.savefig('confusion_matrix_LSTM_theta.png',bbox_inches='tight')

In [ ]:
# test metrics
probab= model.predict(x_test, verbose=2,batch_size=32)
print(f'shape: {probab.shape}')

# make predictions and compute confusion matrix

def predict_classes(probab, batch_size=32, verbose=1):
    if probab.shape[-1] > 1:
        return probab.argmax(axis=-1)
    else:
        return (probab > 0.5).astype('int32')
y_true_test = y_test
y_pred_test = predict_classes(probab)


In [ ]:
# Plot non-normalized confusion matrix
## CM WITH 30 EPOCHS
class_names=['Healthy','Stressed']
plt.figure()
plot_confusion_matrix(cm_test, classes=class_names,
                      title='Confusion matrix')
plt.show()